In [24]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gmaps
import requests
import time
from config_googlemaps import gkey

In [25]:
data_2015 = "chronicle_data2015.csv"
data_2016 = "chronicle_data2016.csv"
data_2017 = "chronicle_data2017.csv"
data_2018 = "chronicle_data2018.csv"
latlong = "statelatlong.csv"

In [26]:
state_latlon = pd.read_csv(latlong, encoding = "ISO-8859-1")
state_latlon

,State_Abbreviation,Latitude,Longitude,State
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California
5,CO,38.997934,-105.550567,Colorado
6,CT,41.518784,-72.757507,Connecticut
7,DE,39.145251,-75.418921,Delaware
8,DC,38.899349,-77.014567,District of Columbia
9,FL,27.975728,-83.833017,Florida


In [27]:
#read to files
data_2015_pd = pd.read_csv(data_2015, encoding = "ISO-8859-1")
data_2016_pd = pd.read_csv(data_2016, encoding = "ISO-8859-1")
data_2017_pd = pd.read_csv(data_2017, encoding = "ISO-8859-1")
data_2018_pd = pd.read_csv(data_2018, encoding = "ISO-8859-1")


In [57]:
frames = [data_2015_pd, data_2016_pd, data_2017_pd, data_2018_pd]

big_df = pd.concat(frames)
big_df.head()


,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [58]:
state_counts = big_df["Where Donor Lives"].value_counts()
state_counts.head()

California    428
New York      268
Texas         197
Florida       164
Illinois      131
Name: Where Donor Lives, dtype: int64

In [59]:
recipient_counts = big_df["Recipient Location"].value_counts()
recipient_counts.head()

California      394
New York        295
Texas           183
Florida         170
Pennsylvania    148
Name: Recipient Location, dtype: int64

In [31]:
renamed_df = big_df.rename(columns={"Where Donor Lives":"State"})
renamed_df.head()

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [32]:
#adding Latitude and Longitude generalized for state locations of donors
coords = pd.merge(renamed_df, state_latlon, how="left", on=["State", "State"])
coords

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value,State_Abbreviation,Latitude,Longitude
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",NY,40.705626,-73.979680
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680
5,2015,Stephen A. Schwarzman,New York,Investments,"Yale U. (New Haven, Conn.)",Connecticut,Colleges and universities,NaN,"$150,000,000",NY,40.705626,-73.979680
6,2015,Donald Sirkin,Washington,Construction,Lighthouse for the Blind and Visually Impaired...,California,Human and social services,Bequest,"$125,000,000",WA,38.899349,-77.014567
7,2015,Samuel Tak Lee,NaN,"Family wealth, Real estate",Massachusetts Institute of Technology (Cambridge),Massachusetts,Colleges and universities,Gift,"$118,000,000",NaN,NaN,NaN
8,2015,David Geffen,California,Entertainment,U. of California at Los Angeles,California,Colleges and universities,NaN,"$100,000,000",CA,37.271875,-119.270415
9,2015,Gordon and Betty Moore,California,Technology,California Institute of Technology (Pasadena),California,Colleges and universities,Pledge Unrestricted,"$100,000,000",CA,37.271875,-119.270415


In [33]:
latlng_df = coords.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
latlng_df.head()

,Year,Donor,State,Donor's Source of Wealth,Recipient,Recipient Location,Cause,Gift type,Gift Value,State_Abbreviation,Lat,Lng
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",NY,40.705626,-73.979680
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680


In [34]:
latlng_df = latlng_df.dropna(how='any')

In [35]:
latlng_df.dtypes

Year                          int64
Donor                        object
State                        object
Donor's Source of Wealth     object
Recipient                    object
Recipient Location           object
Cause                        object
Gift type                    object
Gift Value                   object
State_Abbreviation           object
Lat                         float64
Lng                         float64
dtype: object

In [36]:
recipient_df = big_df.rename(columns={"Recipient Location":"State"})
recipient_df.head()

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000"
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000"
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000"
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000"
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000"


In [37]:
#adding Latitude and Longitude generalized for state locations of recipients
recipient_coords = pd.merge(recipient_df, state_latlon, how="left", on=["State", "State"])
recipient_coords

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value,State_Abbreviation,Latitude,Longitude
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",MA,42.062940,-71.718067
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680
5,2015,Stephen A. Schwarzman,New York,Investments,"Yale U. (New Haven, Conn.)",Connecticut,Colleges and universities,NaN,"$150,000,000",CT,41.518784,-72.757507
6,2015,Donald Sirkin,Washington,Construction,Lighthouse for the Blind and Visually Impaired...,California,Human and social services,Bequest,"$125,000,000",CA,37.271875,-119.270415
7,2015,Samuel Tak Lee,NaN,"Family wealth, Real estate",Massachusetts Institute of Technology (Cambridge),Massachusetts,Colleges and universities,Gift,"$118,000,000",MA,42.062940,-71.718067
8,2015,David Geffen,California,Entertainment,U. of California at Los Angeles,California,Colleges and universities,NaN,"$100,000,000",CA,37.271875,-119.270415
9,2015,Gordon and Betty Moore,California,Technology,California Institute of Technology (Pasadena),California,Colleges and universities,Pledge Unrestricted,"$100,000,000",CA,37.271875,-119.270415


In [38]:
recipient_latlng_df = recipient_coords.rename(columns={"Latitude":"Lat", "Longitude":"Lng"})
recipient_latlng_df.head()

,Year,Donor,Where Donor Lives,Donor's Source of Wealth,Recipient,State,Cause,Gift type,Gift Value,State_Abbreviation,Lat,Lng
0,2015,John Santikos,Texas,Real estate,San Antonio Area Foundation,Texas,Community foundations,Bequest,"$605,000,000",TX,31.169336,-100.076842
1,2015,John A. Paulson,New York,Finance,"Harvard U. (Cambridge, Mass.)",Massachusetts,Colleges and universities,Unrestricted,"$400,000,000",MA,42.062940,-71.718067
2,2015,Stefan Edlis and Gael Neeson,Illinois,Manufacturing,Art Institute of Chicago,Illinois,Arts,Noncash Gift,"$400,000,000",IL,39.739318,-89.504139
3,2015,William H. Scheide,New Jersey,Family wealth,Princeton U. (N.J.),New Jersey,Colleges and universities,Noncash Gift,"$300,000,000",NJ,40.143006,-74.731116
4,2015,David H. Koch,New York,Oil,Memorial Sloan-Kettering Cancer Center (New York),New York,Health,Pledge,"$150,000,000",NY,40.705626,-73.979680


In [39]:
recipient_latlng_df = latlng_df.dropna(how='any')

In [40]:
recipient_latlng_df.dtypes

Year                          int64
Donor                        object
State                        object
Donor's Source of Wealth     object
Recipient                    object
Recipient Location           object
Cause                        object
Gift type                    object
Gift Value                   object
State_Abbreviation           object
Lat                         float64
Lng                         float64
dtype: object

In [50]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [51]:
# Store 'Lat' and 'Lng' into  locations 
locations = latlng_df[["Lat", "Lng"]].astype(float)

#donation_amount = latlng_df["Gift Value"].astype(object)



In [52]:
# map based on the donors state of residence
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                dissipating=False, max_intensity=100, point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius =1 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [55]:
# Store 'Lat' and 'Lng' into  locations 
recipient_locations = recipient_latlng_df[["Lat", "Lng"]].astype(float)

#donation_amount = recipient_latlng_df["Gift Value"].astype(object)




In [56]:
#map based on recipient locations
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(recipient_locations, 
                                dissipating=False, max_intensity=100, point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius =1 
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))